# Hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
number_of_folds = 5 # this sets global setting of which how many bootstraps to use
repeats = 10
#first round of optimization
lgb_params = {"bagging_fraction": 0.9522534844058304, 
              "boosting_type": "dart", 
              "objective": "regression",
              "feature_fraction": 0.42236910941558053, 
              "lambda_l1": 0.020847266580277746, 
              "lambda_l2": 2.8448564854773326, 
              "learning_rate": 0.11484015430016059, 
              "max_depth": 3, 
              "max_leaves": 35, 
              "min_data_in_leaf": 9,
              "num_iterations": 150
             }
debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.selection import ShapSelector
from yspecies.tuning import Tune
from yspecies.models import ResultsCV, CrossValidator
import optuna
from optuna import Study, Trial

In [5]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [6]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [7]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(445, 12340)",12340,39,445,"(12340, 2)","(40, 19)"


## Checking that crossvalidation works ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [8]:
partition_params = PartitionParameters(number_of_folds, 1, 2, [],  42)


In [9]:
partition_cv_pipe = Pipeline([
    ('partitioner', DataPartitioner()),
    ('prepare_for_partitioning', TupleWith(lgb_params)),
    ('crossvalidator', CrossValidator())
]
)

In [10]:
repeats = 3
repeated_cv_pipe =  Repeat(partition_cv_pipe, repeats, lambda x,i: (x[0], replace(x[1], seed = i)))

In [11]:
cv_pipeline =  Pipeline([
    ('extractor', DataExtractor()),
    ('prepare_for_partitioning', TupleWith(partition_params)), # to extract the data required for ML from the dataset
    ("partition_cv", repeated_cv_pipe)
    ]
    )

## Setting up features to select ##

In [12]:
selection = select_lifespan = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"])

In [13]:
select_lifespan = selection
select_mass = replace(selection, to_predict = "mass_g")
select_gestation = replace(selection, to_predict = "gestation")
select_mtgc = replace(selection, to_predict = "mtgc")

## Checking cross-validation ##

In [14]:
cv_res = cv_pipeline.fit_transform((data, select_lifespan))
ResultsCV.take_best(cv_res)

Found `num_iterations` in params. Will use it instead of argument
Early stopping is not available in dart mode


[150]	cv_agg's l1: 2.32733 + 0.158731	cv_agg's l2: 33.7984 + 6.72072	cv_agg's huber: 1.75966 + 0.140074
[150]	cv_agg's l1: 11.2941 + 7.39369	cv_agg's l2: 190.391 + 150.911	cv_agg's huber: 9.79839 + 6.60662
[150]	cv_agg's l1: 5.903 + 7.23029	cv_agg's l2: 111.941 + 158.024	cv_agg's huber: 4.97673 + 6.4728


1.4383286031036224

In [15]:
[c.last("huber")for c in cv_res]

[1.7596586313262346, 9.798388686288867, 4.976725120564676]

# Optimization #

In [16]:
mean_repeated_cv = Pipeline([
    ("repeated_cross_validation", repeated_cv_pipe),
    ("collect_mean", Collect(fold=lambda results: np.array([r.last("huber") for r in results]).mean()) )
])

In [17]:
mean_cv_pipeline =  Pipeline([
    ('extractor', DataExtractor()),
    ('prepare_for_partitioning', TupleWith(partition_params)), # to extract the data required for ML from the dataset
    ("mean_cv", mean_repeated_cv) ]
    )

In [23]:
r = mean_cv_pipeline.fit_transform((data, select_lifespan))
r

[150]	cv_agg's l1: 2.42168 + 0.126173	cv_agg's l2: 33.4596 + 2.38675	cv_agg's huber: 1.84686 + 0.108799
[150]	cv_agg's l1: 2.41872 + 0.327256	cv_agg's l2: 33.7097 + 7.41532	cv_agg's huber: 1.84488 + 0.277623
[150]	cv_agg's l1: 2.44422 + 0.244433	cv_agg's l2: 33.2034 + 2.2935	cv_agg's huber: 1.86806 + 0.216065


1.8532666599348968

In [18]:
url = f'sqlite:///' +str((locations.metrics.lifespan / "study.sqlite").absolute())
print('loading (if exists) study from '+url)
storage = optuna.storages.RDBStorage(
    url=url
    #engine_kwargs={'check_same_thread': False}
)

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/metrics/lifespan/study.sqlite


In [19]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 4.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 4.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [24]:
tune_pipeline =  Pipeline([
    ('extractor', DataExtractor()),
    ('prepare_for_partitioning', TupleWith(partition_params)), # to extract the data required for ML from the dataset
    ("partition_cv", Tune(mean_repeated_cv, n_trials = 2, parameters_space = optimization_parameters))
    ]
    )

In [26]:
best = tune_pipeline.fit_transform((data, select_lifespan))
best

[W 2020-08-13 07:11:44,512] Trial 1 failed because of the following error: TypeError('replace() should be called on dataclass instances')
Traceback (most recent call last):
  File "/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/optuna/study.py", line 709, in _run_trial
    result = func(trial)
  File "../yspecies/tuning.py", line 107, in objective
    result = self.transformer.fit_transform((data, params))
  File "/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/sklearn/pipeline.py", line 367, in fit_transform
    Xt = self._fit(X, y, **fit_params_steps)
  File "/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/sklearn/pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/sklearn/pipeline.py", line 740, in _fit_transfor

TypeError: replace() should be called on dataclass instances